| Kolom                    | Interpretasi                                  |
| ------------------------ | --------------------------------------------- |
| Age                      | Usia                                          |
| Attrition                | Tidak bekerja lagi                            |
| BusinessTravel           | Frekuensi bepergian (travelling)              |
| DailyRate                | Upah harian                                   |
| Department               | Departmen pada pekerjaan                      |
| DistanceFromHome         | Jarak dari rumah ke tempat kerja (KM)         |
| Education                |  1 'Below College' 2 'College' 3 'Bachelor' 4 'Master' 5 'Doctor' |
| EducationField           | Bidang pendidikan                             |
| EmployeeCount            |                                               |
| EmployeeNumber           |                                               |
| EnvironmentSatisfaction  | Kepuasan terhadap lingkungan kerja => 1 'Low' 2 'Medium' 3 'High' 4 'Very High'            |
| Gender                   | Jenis kelamin                                 |
| HourlyRate               | Upah perjam                                   |
| JobInvolvement           | Tingkat keterlibatan kerja => 1 'Low' 2 'Medium' 3 'High' 4 'Very High'                   |
| JobLevel                 | Level kesulitan pekerjaan                     |
| JobRole                  | Posisi Pekerjaan                              |
| JobSatisfaction          | Kepuasan terhadap pekerjaan => 1 'Low' 2 'Medium' 3 'High' 4 'Very High'                  |
| MaritalStatus            | Status Perkawinan                             |
| MonthlyIncome            | Income Bulanan                                |
| MonthlyRate              | Upah Bulanan                                  |
| NumCompaniesWorked       | Banyaknya perusahaan tempat bekerja           |
| Over18                   | Apakah usianya > 18?                          |
| OverTime                 | Apakah jam kerja overtime?                    |
| PercentSalaryHike        | Persentase kenaikan gaji                      |
| PerformanceRating        | Rating performa kerja  => 1 'Low' 2 'Good' 3 'Excellent' 4 'Outstanding'                       |
| RelationshipSatisfaction | Kepuasan terhadap relasi di tempat kerja => 1 'Low' 2 'Medium' 3 'High' 4 'Very High'      |
| StandardHours            | Standard jam kerja                            |
| StockOptionLevel         |                                               |
| TotalWorkingYears        | Lama bekerja (Tahun)                          |
| TrainingTimesLastYear    | Jumlah training tahun kemarin                 |
| WorkLifeBalance          | Keseimbangan kerja => 1 'Bad' 2 'Good' 3 'Better' 4 'Best'                           |
| YearsAtCompany           | Lama bekerja pada perusahaan                  |
| YearsInCurrentRole       | Lama bekerja pada posisi ini                  |
| YearsSinceLastPromotion  | Lama berkerja sejak kenaikan jabatan terakhir |
| YearsWithCurrManager     | Lama bekerja dengan manager saat ini          |

# __SUPERVISED LEARNING__

In [16]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import rcParams

rcParams['figure.figsize'] = (10,7)
rcParams['lines.linewidth'] = 2.5
rcParams['xtick.labelsize'] = 'x-large'
rcParams['ytick.labelsize'] = 'x-large'

sns.set_theme(style='darkgrid')
sns.set(font_scale=1.2, palette='Set1')

## __MODELLING__

### __SPLIT DATA__

Kami akan melakukan split data dengan menggunakan train_test_split dengan perbandingan 70:30

In [17]:
df = pd.read_csv('../Stage 2/Preprocessed.csv')

#pisahkan target dan fitur
x = df.drop('Attrition', axis=1)
y = df['Attrition']

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(x,y, test_size=0.3, stratify=y, random_state=7)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((1726, 20), (740, 20), (1726,), (740,))

 - Kasus False Negative

 Karyawan yang diprediksi akan melakukan resign tapi data aktualnya tidak resign, hal ini jika dilakukan "perlakuan khusus" akan membuat cost untuk hal ini menjadi membesar tanpa hasil apapun karena memang karyawan tidak berencana utk resign

 - Kasus False Positive

 Karyawan yang diprediksi tidak akan resign, tapi data aktualnya resign, hal ini jika dilakukan "perlakuan khusus" akan membuat company kehilangan karyawan yang potensial, sehingga membuat company mengalami kerugian karena juga harus melakukan rekrutmen yang membutuhkan biaya

Metriks yang kita gunakan dalam case ini adalah roc_auc, karena kita sama sama mementingkan FN dan FP, selain itu dataset ini juga termasuk imbalance 

In [25]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.model_selection import cross_validate

def eval_classification(model):
        y_pred = model.predict(X_test)
        y_pred_train = model.predict(X_train)
        y_pred_proba = model.predict_proba(X_test)
        y_pred_proba_train = model.predict_proba(X_train)

        print("Accuracy (Test Set): %.2f" % accuracy_score(y_test, y_pred))
        print("Precision (Test Set): %.2f" % precision_score(y_test, y_pred))
        print("Recall (Test Set): %.2f" % recall_score(y_test, y_pred))
        print("F1-Score (Test Set): %.2f" % f1_score(y_test, y_pred))

        print("\nroc_auc (train-proba): %.2f" % roc_auc_score(y_train, y_pred_proba_train[:, 1]))
        print("roc_auc (test-proba): %.2f" % roc_auc_score(y_test, y_pred_proba[:, 1]))

        score = cross_validate(model, x, y, cv=5, scoring='roc_auc', return_train_score=True)
        print('\nroc_auc (crossval train): '+ str(round(score['train_score'].mean(),2)))
        print('roc_auc (crossval test): '+ str(round(score['test_score'].mean(),2)))

### __LOGISTIC REGRESSION__

Kami menggunakan algoritma logistik regression karena algoritma ini termasuk yang simple dan komputasinya cepat, sehingga bisa dijadikan sebagai base model untuk melihat seberapa bagus performa model

In [26]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(max_iter=110)
lr.fit(X_train, y_train)

eval_classification(lr)

Accuracy (Test Set): 0.86
Precision (Test Set): 0.88
Recall (Test Set): 0.84
F1-Score (Test Set): 0.86

roc_auc (train-proba): 0.91
roc_auc (test-proba): 0.93

roc_auc (crossval train): 0.92
roc_auc (crossval test): 0.91


Karena score pada model Logistic Regression sudah sangat bagus dan tidak adanya indikasi overfitting/underfitting, maka model ini tidak perlu lagi dilakukan tuning

### __K-NEAREST NEIGBORS__

Kami menggunakan algoritma KNN karena dataset ini bersifat non linear, walaupun KNN termasuk algoritma yg membutuhkan komputasi banyak, dataset ini hanya memiliki 2466 baris dan 20 kolom, sehingga masih tidak terlalu berat untuk digunakan

In [27]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier()
knn.fit(X_train, y_train)

eval_classification(knn)

Accuracy (Test Set): 0.88
Precision (Test Set): 0.85
Recall (Test Set): 0.93
F1-Score (Test Set): 0.89

roc_auc (train-proba): 0.97
roc_auc (test-proba): 0.94

roc_auc (crossval train): 0.97
roc_auc (crossval test): 0.93
